In [1]:
import sys
import pandas as pd
import os
from sklearn.metrics import accuracy_score,precision_score,recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from feature_engine import imputation as mdi
from feature_engine import encoding as ce
# import mean_median2 as mm
import warnings
warnings.filterwarnings('ignore')


def get_path():
    cur_path = os.getcwd()
    parent_path = os.path.dirname(cur_path)
    return cur_path, parent_path


def file_path(data_path, file):
    return os.path.abspath(os.path.join(data_path, f'{file}'))


def df_write(data_path, df, file):
    df = df.copy()
    df.to_csv(os.path.abspath(os.path.join(data_path, file)), index=False)


def split_train_test(df, configs):
    df = df.copy()
    X = df.drop(columns=configs['y_col'][0])
    y = df[configs['y_col'][0]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=configs['y_col'][0])
    return X_train, X_test, y_train, y_test


def model_selection(option='logic'):
    if option == 'light':
        return lgb.LGBMClassifier(random_state=0)
    else:
        return LogisticRegression(random_state=0)


def read_data(configs):
    if configs['date_col'][0] == ' ':
        df = pd.read_csv(configs['file_name'][0])
    else:
        df = pd.read_csv(configs['file_name'][0], parse_dates=configs['date_col'])

    if configs['remove_col'][0] == ' ':
        pass
    else:
        if configs['remove_col'][0] in df.columns.to_list():
            df = df.drop(configs['remove_col'][0], axis=1)

    return df


def y_label_enc(df, configs):
    df = df.copy()
    Y_col = configs['y_col'][0]
    if df[Y_col].isnull().any():
        Y_null = True
    else:
        Y_null = False
    labeler = LabelEncoder()
    df[Y_col] = labeler.fit_transform(df[Y_col])
    return df, Y_null


def organize_data(df, configs, y_null):
    df = df.copy()
    cols = df.columns.to_list()
    null_threshhold_cols = []
    no_null_cols = []
    date_time = configs['date_col']
    Y_col = configs['y_col'][0]

    for col in cols:
        null_mean = df[col].isnull().mean()
        if null_mean >= configs['null_threshhold'][0]:
            null_threshhold_cols.append(col)
        if null_mean == 0:
            no_null_cols.append(col)

    cols_stayed = [item for item in cols if item not in null_threshhold_cols]
    data = df[cols_stayed].copy()

    # numerical: discrete vs continuous
    discrete = [var for var in cols_stayed if
                data[var].dtype != 'O' and var != Y_col and var not in date_time and data[var].nunique() < 10]
    continuous = [var for var in cols_stayed if
                  data[var].dtype != 'O' and var != Y_col and var not in date_time and var not in discrete]

    # categorical
    categorical = [var for var in cols_stayed if data[var].dtype == 'O' and var != Y_col]

    print('There are {} date_time variables'.format(len(date_time)))
    print('There are {} discrete variables'.format(len(discrete)))
    print('There are {} continuous variables'.format(len(continuous)))
    print('There are {} categorical variables'.format(len(categorical)))

    if y_null:
        data = data[data[Y_col] != data[Y_col].max()].copy()
    else:
        data = data.copy()

    return data, discrete, continuous, categorical


def make_train_test(df, configs):
    df = df.copy()
    X = df.drop(columns=configs['y_col'][0])
    y = df[configs['y_col'][0]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=configs['test_size'][0], random_state=0, stratify=y)
    return X_train, X_test, y_train, y_test


def make_imputer_pipe(continuous, discrete, categorical, null_impute_type=None):
    numberImputer = continuous + discrete
    categoricalImputer = categorical
    
    if null_impute_type is None:
        pipe = []
    else:
        if (len(numberImputer) > 0):
            pipe = Pipeline([
                ("imputer", mdi.MeanMedianImputer(imputation_method=null_impute_type, variables=numberImputer),)
            ])        
    return pipe


def do_imputation(df, configs, pipe):
    if pipe != []:
        df = df.copy()
        X_train, X_test, y_train, y_test = make_train_test(df, configs)
        
        pipe.fit(X_train, y_train)
        
        X_train = pipe.transform(X_train)
        X_test = pipe.transform(X_test)

        X_train[configs['y_col'][0]] = y_train        
        X_train['split'] = 'train'
        X_test[configs['y_col'][0]] = y_test
        X_test['split'] = 'test'        
        return pd.concat([X_train, X_test]).reset_index(drop=True)
    else:
        print('no pipe applied')
        return df    


def do_train(X_train, X_test, y_train, y_test, option):
    X_train, X_test, y_train, y_test = X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()
    model = model_selection(option)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred, option)


def min_max_scale(df):
    df = df.copy()
    scaler = MinMaxScaler()
    scaler.fit(df)
    return scaler.transform(df)


def metrics(y_test, pred, option):
    y_test = y_test.copy()
    pred = pred.copy()
    accuracy = round(accuracy_score(y_test, pred), 2)
    precision = round(precision_score(y_test, pred), 2)
    recall = round(recall_score(y_test, pred), 2)
    f1 = round(f1_score(y_test, pred), 2)
    print(option, "f1 점수:", f1, "정확도:", accuracy, "정밀도:", precision, "재현율:", recall)
    print(confusion_matrix(y_test, pred))

In [2]:
# arv 예1: credit argumet_credit.xlsx
# arv 예2: metro argumet_metro.xlsx

try:
    folder_name = 'metro'
    config_file_name = 'argumet_metro.xlsx'
    cur_path = os.getcwd()
    parent = os.path.abspath(os.path.join(cur_path, os.pardir))
    config_file = os.path.join(parent, os.path.join('config', f'{config_file_name}'))
    configs = pd.read_excel(config_file, header=None).set_index(0).T
    configs = configs.to_dict('list')
    ori_file_name = configs['file_name'][0]
    configs['file_name'][0] = os.path.join(parent, os.path.join(f'data/{folder_name}', configs['file_name'][0]))
    df_initial = read_data(configs)

#     # 전처리 저장 경로 정의
#     dest_path = os.path.join(parent, os.path.join('data_preprocessed', f'{folder_name}'))
#     dest_path = os.path.join(parent, os.path.join(f'{dest_path}/imputed', f'draft_{ori_file_name}'))       

#     # 오리지널 데이터셋 저장
#     df_initial.to_csv(dest_path, index=False)        

    # 1. Label 칼럼 인코딩   
    df, y_null = y_label_enc(df_initial, configs)

    # 2. discrete, continuous, categorical 구분작업
    df_organized, discrete, continuous, categorical = organize_data(df, configs, y_null)

    # null_impute_types 정의
    null_impute_types = ['median', 'mean'] #, 'max', 'min']

    for null_impute_type in null_impute_types:
        # 3. pipe 작업
        pipe = make_imputer_pipe(discrete, continuous, categorical, null_impute_type='median')

        # 4. imputation with train/test split
        df_imputed = do_imputation(df_organized, configs, pipe)

#         # 5. 전처리 셋 저장    
#         dest_path = os.path.join(parent, os.path.join('data_preprocessed', f'{folder_name}'))
#         dest_path = os.path.join(parent, os.path.join(f'{dest_path}/imputed', f'{null_impute_type}_imputed_{ori_file_name}'))
#         df_imputed.to_csv(dest_path, index=False)

#         # 6. 스케일링 작업 및 저장
#         # 6.1 X_train 스케일링
#         Y_COL = configs['y_col'][0]
#         con = df_imputed['split']=='train'
#         X_train = df_imputed[con].reset_index(drop=True)
#         y_train = X_train[Y_COL].reset_index(drop=True)
#         X_train = X_train.drop(columns=[Y_COL, 'split'])                        
#         X_train_scaled = min_max_scale(X_train)

#         # 6.2 X_test 스케일링
#         con = df_imputed['split']=='test'
#         X_test = df_imputed[con].reset_index(drop=True)
#         y_test = X_test[Y_COL].reset_index(drop=True)
#         X_test = X_test.drop(columns=[Y_COL, 'split'])                        
#         X_test_scaled = min_max_scale(X_test)

#         # 6.3 data frame으로 변환
#         xtrains = pd.DataFrame(data=X_train_scaled, columns=X_train.columns)
# #         xtrains['split'] = 'train'
# #         xtrains[Y_COL] = y_train
#         xtests = pd.DataFrame(data=X_test_scaled, columns=X_test.columns)
# #         xtests['split'] = 'test'
# #         xtests[Y_COL] = y_test
# #         df_scaled = pd.concat([xtrains, xtests], ignore_index=True)
#         # 6.4 scaling 저장
#         1/0
#         dest_path = os.path.join(parent, os.path.join('data_preprocessed', f'{folder_name}'))
#         dest_path = os.path.join(parent, os.path.join(f'{dest_path}/scaled', f'{null_impute_type}_Scaled_Xtrain_{ori_file_name}'))
#         df_scaled.to_csv(dest_path, index=False)

#     print('Completed.')

except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print('비정상종료', e)
    print(exc_type, exc_tb.tb_lineno)



There are 1 date_time variables
There are 8 discrete variables
There are 7 continuous variables
There are 0 categorical variables


In [3]:
df = pd.read_csv(r'C:\Users\Digitalship_PC\pydev\digitalship\data\metro\MetroPT3(AirCompressor) _ori.csv')
df['y'].value_counts(True)

0    0.96481
1    0.03519
Name: y, dtype: float64

In [129]:
from imblearn.over_sampling import SMOTE
X = df.drop(columns=['timestamp', 'y'])
y = df['y']
oversample = SMOTE(sampling_strategy=0.15, random_state=0)
X, y = oversample.fit_resample(X, y)
df0 = pd.concat([X,y], 1)
df0['y'].value_counts(True)

0    0.869566
1    0.130434
Name: y, dtype: float64

In [130]:
times = pd.date_range('2019-09-01', periods=df0.shape[0], freq='1min')
cols = ['timestamp'] + list(df0.columns)
df0['timestamp'] = times
df0 = df0[cols]
df0

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,y
0,2019-09-01 00:00:00,-0.012000,9.358000,9.340000,-0.024000,9.358000,53.600000,0.040000,1,0,1,1,0,1,1,1,0
1,2019-09-01 00:01:00,-0.014000,9.348000,9.332000,-0.022000,9.348000,53.675000,0.040000,1,0,1,1,0,1,1,1,0
2,2019-09-01 00:02:00,-0.012000,9.338000,9.322000,-0.022000,9.338000,53.600000,0.042500,1,0,1,1,0,1,1,1,0
3,2019-09-01 00:03:00,-0.012000,9.328000,9.312000,-0.022000,9.328000,53.425000,0.040000,1,0,1,1,0,1,1,1,0
4,2019-09-01 00:04:00,-0.012000,9.318000,9.302000,-0.022000,9.318000,53.475000,0.040000,1,0,1,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163422,2021-11-16 22:22:00,-0.010337,9.755010,9.743010,-0.016000,9.755010,66.566841,3.547184,1,0,1,1,0,1,1,1,1
1163423,2021-11-16 22:23:00,8.891382,8.731644,-0.009044,1.974606,8.731732,72.336953,5.811195,0,1,0,0,0,1,1,1,1
1163424,2021-11-16 22:24:00,8.153199,7.959596,-0.008801,2.112397,7.961199,75.200000,5.533998,0,1,0,0,0,1,1,1,1
1163425,2021-11-16 22:25:00,8.205739,7.987043,-0.008348,2.132348,7.989391,75.004348,5.673804,0,1,1,0,0,1,1,1,1


In [117]:
# df1 = df0.copy()
# dfy = df1[df1.y==1]
# dfn = df1[df1.y==0]

In [118]:
# df1.loc[df1.query('y == 1').sample(frac=.1).index, 'y'] = 0
# dfn.loc[ dfn.query('y == 0').sample(frac=.1).index, 'y'] = 1

In [119]:
# df1 = pd.concat([dfy,dfn]).sort_values('timestamp')
# df1

In [131]:
## 널 만들기
cols = ['TP2', 'TP3', 'H1', 'Reservoirs', 'Oil_temperature', 'Motor_current']
# df = pd.read_csv(r'C:\Users\Digitalship_PC\pydev\digitalship\data\metro\MetroPT3(AirCompressor) _ori.csv')
df2 = df0.copy()
for i, col in enumerate(cols):  
    df2.loc[df2.sample(frac=0.05).index, col] = pd.np.nan
df2.to_csv(r'C:\Users\Digitalship_PC\pydev\digitalship\data\metro\MetroPT3(AirCompressor).csv', index=False)

In [4]:
## 널 만들기
cols = ['TP2', 'TP3', 'H1', 'Reservoirs']#, 'Oil_temperature'] #, 'Motor_current']
df = pd.read_csv(r'C:\Users\Digitalship_PC\pydev\digitalship\data\metro\MetroPT3(AirCompressor) _ori.csv')
df1 = df.copy()
for i, col in enumerate(cols):
    df1.loc[df1.sample(frac=0.15).index, col] = pd.np.nan
df1.to_csv(r'C:\Users\Digitalship_PC\pydev\digitalship\data\metro\MetroPT3(AirCompressor).csv', index=False)

# dfmin = pd.read_csv(r'C:\Users\Digitalship_PC\pydev\digitalship\data_preprocessed\metro\imputed\min_imputed_MetroPT3(AirCompressor).csv')
# dfmax = pd.read_csv(r'C:\Users\Digitalship_PC\pydev\digitalship\data_preprocessed\metro\imputed\max_imputed_MetroPT3(AirCompressor).csv')
# (dfmin == dfmax).any()

In [83]:
dfmin = pd.read_csv(r'C:\Users\Digitalship_PC\pydev\digitalship\data_preprocessed\metro\imputed\imputed_MetroPT3(AirCompressor)_min.csv')
dfmax = pd.read_csv(r'C:\Users\Digitalship_PC\pydev\digitalship\data_preprocessed\metro\imputed\imputed_MetroPT3(AirCompressor)_max.csv')

,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,y,split
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1048571,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True
1048572,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1048573,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [87]:
(dfmin == dfmax).any().any()

True

In [79]:
d = pd.read_csv(r'C:\Users\Digitalship_PC\pydev\digitalship\data\metro\MetroPT3(AirCompressor).csv')
d

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,y
0,2020-02-01 0:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1,0,1,1,0,1,1,1,0
1,2020-02-01 0:00,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1,0,1,1,0,1,1,1,0
2,2020-02-01 0:00,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1,0,1,1,0,1,1,1,0
3,2020-02-01 0:00,-0.012,NaN,NaN,NaN,9.328,53.425,0.0400,1,0,1,1,0,1,1,1,0
4,2020-02-01 0:00,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1,0,1,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2020-06-26 13:13,-0.010,9.382,9.370,-0.018,9.384,68.300,3.7425,1,0,1,1,0,1,1,1,0
1048571,2020-06-26 13:14,-0.010,9.366,9.356,-0.018,9.368,68.050,3.7625,1,0,1,1,0,1,1,1,0
1048572,2020-06-26 13:14,-0.012,9.356,9.346,-0.018,9.358,67.825,3.6550,1,0,1,1,0,1,1,1,0
1048573,2020-06-26 13:14,-0.012,9.344,NaN,-0.018,9.344,67.650,3.7600,1,0,1,1,0,1,1,1,0


In [80]:
con1 = d['TP2']==-0.012
con2 = d['Reservoirs']==9.328
con3 = d['Oil_temperature']==53.425
d[con1&con2&con3]

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,y
3,2020-02-01 0:00,-0.012,NaN,NaN,NaN,9.328,53.425,0.04,1,0,1,1,0,1,1,1,0


In [88]:
con1 = dfmax['TP2']==-0.012
con2 = dfmax['Reservoirs']==9.328
con3 = dfmax['Oil_temperature']==53.425
dfmax[con1&con2&con3]

,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,y,split
452767,-0.012,10.302,10.288,9.844,9.328,53.425,0.04,1,0,1,1,0,1,1,1,0,train


In [89]:
con1 = dfmin['TP2']==-0.012
con2 = dfmin['Reservoirs']==9.328
con3 = dfmin['Oil_temperature']==53.425
dfmin[con1&con2&con3]

,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,y,split
452767,-0.012,0.806,-0.036,-0.032,9.328,53.425,0.04,1,0,1,1,0,1,1,1,0,train


In [93]:
dfmin.iloc[452767] == dfmax.iloc[452767]

TP2                 True
TP3                False
H1                 False
DV_pressure        False
Reservoirs          True
Oil_temperature     True
Motor_current       True
COMP                True
DV_eletric          True
Towers              True
MPG                 True
LPS                 True
Pressure_switch     True
Oil_level           True
Caudal_impulses     True
y                   True
split               True
Name: 452767, dtype: bool